In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import keras
from keras import layers, models, utils, backend
from keras.models import Sequential
from keras.layers import Dense
#from keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
#from mlxtend.plotting import plot_decision_regions
from seaborn import countplot



# Załadowanie danych z .parquet file
df = pd.read_parquet('data_pruned_normalized.parquet')

# Identyfikacja kolumn feature (wszystkich oprócz AttackType)
label_column = 'Label'
feature_columns = [col for col in df.columns if col != label_column]

# Ekstrakcja features i labels
X = df[feature_columns]
y = df[label_column]

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Podział danych na zestawy training (70%) i testing (30%)
X_train_val, X_final_test, y_train_val, y_final_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Dalszy podział danych treningowych na zestaw training (70%) i validation (30%)
X_train, X_test, y_train, y_test = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=42)

# Wyświetlenie próbek poszczególnych danych
print("Training data sample (X_train):")
print(X_train[:5])
print("\nTraining labels sample (y_train):")
print(y_train[:5])
print("Test data sample (X_test):")
print(X_test[:5])
print("\nTest labels sample (y_test):")
print(y_test[:5])
print("Final test data sample (X_final_test):")
print(X_final_test[:5])
print("\nFinal test labels sample (y_final_test):")
print(y_final_test[:5])



Training data sample (X_train):
         Dst Port  Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
1350601  0.006715   1.132148e-02      0.000665      0.000261         0.000042   
826622   0.000244   0.000000e+00      0.000000      0.000052         0.000000   
728513   0.000244   0.000000e+00      0.000000      0.000052         0.000000   
915185   0.000259   2.936533e-03      0.002328      0.001147         0.000222   
533349   0.001145   6.916667e-07      0.000111      0.000000         0.000000   

         Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  Fwd Pkt Len Std  \
1350601         0.001598              0.0          0.003146         0.002675   
826622          0.000000              0.0          0.000000         0.000000   
728513          0.000000              0.0          0.000000         0.000000   
915185          0.009932              0.0          0.005346         0.007492   
533349          0.000000              0.0          0.000000         0.000000   



In [2]:
## Po zmianach

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss

# Ekstrakcja features z finalnych testowych danych
X_final_test_features = X_final_test[feature_columns]

# Standaryzacja features
scaler = StandardScaler()
X_final_test_features_stand = scaler.fit_transform(X_final_test_features)
X_final_test_features_transf = scaler.transform(X_final_test_features_stand)

# Inicjalizacja i trenowanie modelu regresji logistycznej
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)

# Ewaluacja precyzji modelu na danych testowych
print("\nEwaluacja testowych danych (X_test, y_test):")
y_pred_test = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_loss = log_loss(y_test, model.predict_proba(X_test))
print(f'binary_accuracy = {test_accuracy * 100}, set loss = {test_loss}')

# Ewaluacja modelu na finalnych danych testowych
y_pred_final_test = model.predict(X_final_test)
final_test_accuracy = accuracy_score(y_final_test, y_pred_final_test)
final_test_loss = log_loss(y_final_test, model.predict_proba(X_final_test))
print(f'binary_accuracy from eval = {final_test_accuracy * 100}, set loss from eval = {final_test_loss}')

# Sprawdzenie predykcji
print("\nPredykcja testowych finalnych danych (X_final_test):")
predictions = model.predict(X_final_test_features_transf)

# Tworzenie dynamicznego mapowania ataków opartego o labelki obecne w dataset (z pliku)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
attack_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}

# Konwersja predykcji na nazwy typów ataków 
predicted_attack_types = [attack_mapping[label] for label in predictions]

# Liczenie ilości ataków dla każdego typu
is_attack_counts = pd.Series(predicted_attack_types).value_counts()

# Pokazywanie ilości ataków
print(is_attack_counts)






D:\Anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Ewaluacja testowych danych (X_test, y_test):
binary_accuracy = 97.15299535932495, set loss = 0.13070726690308224
binary_accuracy from eval = 97.1465747942596, set loss from eval = 0.1284539835559186

Predykcja testowych finalnych danych (X_final_test):


D:\Anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


2    161908
0    129980
1    107614
3     47785
Name: count, dtype: int64


In [16]:
#####

# funkcja sigmoid logistyczna, która ogranicza wartości od 0 do 1. Ta funkcja okresla prawdopodobieństwo, że punkt nalezy do klasy lub nie
# na wejscie są tylko 2 warstwy (45 kolumn)

model = keras.Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(1, activation='softplus'))


# Kompilowanie modelu  wraz z optymalizatorem: 
# - metoda stochastycznego spadku wzdłóź gradientu (sgd), lub inna metoda na większe ilości danych (adam)
# - funkcja obliczająca straty i koszty (binary_crossentropy)
# - metryka śledząca postępy uczenia binary_accuracy
optimizer_sgd = SGD(learning_rate=0.001,decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer_sgd, loss='binary_crossentropy', metrics=['binary_accuracy'])

print("\nTrenowanie modelu na bazie treningowych danych (X_train, y_train):")
model.fit(
   x=X_train,
   y=y_train,
   shuffle=True,
   epochs=5,
   batch_size=16
)

#Ewaluacja precyzji modelu
print("\nEwaluacja testowych danych (X_test, y_test):")
loss, binary_accuracy = model.evaluate(x=X_test, y=y_test)
print(f'binary_accuracy = {binary_accuracy*100}, set loss = {loss}')

eval = model.evaluate(x=X_final_test, y=y_final_test)
print(f'binary_accuracy from eval = {eval[1]*100}, set loss from eval = {eval[0]}')

# Ekstrakcja features z finalnych testowych danych
X_final_test_features = X_final_test[feature_columns]

# Standaryzacja features
scaler = StandardScaler()
X_final_test_features_stand = scaler.fit_transform(X_final_test_features)
X_final_test_features_transf = scaler.transform(X_final_test_features_stand)

# Sprawdzenie predykcji
print("\nPredykcja testowych finalnych danych (X_final_test):")
predictions = model.predict(X_final_test_features_transf)


# Tworzenie dynamicznego mapowania ataków opartego o labelki obecne w dataset (z pliku)
attack_mapping = {index: label for index,
                  label in enumerate(label_encoder.classes_)}

# Konwersja predykcji na nazwy typów ataków 
predicted_attack_types = [attack_mapping[label] for label in y_final_test]

# Liczenie ilości ataków dla każdego typu
is_attack_counts = pd.Series(predicted_attack_types).value_counts()

# Pokazywanie ilości ataków
print(is_attack_counts)


#####

c:\Users\pkrolak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\pkrolak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(



Trenowanie modelu na bazie treningowych danych (X_train_new, y_train_new):
Epoch 1/5
32108/32108 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - binary_accuracy: 0.4398 - loss: 4.5534
Epoch 2/5
32108/32108 ━━━━━━━━━━━━━━━━━━━━ 77s 2ms/step - binary_accuracy: 0.4399 - loss: 4.6266
Epoch 3/5
 5805/32108 ━━━━━━━━━━━━━━━━━━━━ 1:01 2ms/step - binary_accuracy: 0.4411 - loss: 4.6005